## Implementing a Perceptron

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

df= pd.read_csv('./day5_titanic/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [2]:
y=df['Survived']
X=df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

In [3]:
X_numeric = X[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
X_categorical = X[['Sex','Embarked']]
X_neither = X['Cabin']
X_onehot = pd.get_dummies(X_categorical,['Sex','Embarked']).astype(int)
X_cabin_nonna = X['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
X_encoded0 = pd.merge(X_numeric, X_cabin_nonna,  left_index=True, right_index=True)
X_encoded1 = pd.merge(X_encoded0, X_onehot,  left_index=True, right_index=True)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded1, y, test_size=0.2, random_state=42, stratify=y
)
from sklearn.impute import SimpleImputer

# Imputer for numeric columns (e.g., Age)
imputer = SimpleImputer(strategy='median')  # or 'mean' if you prefer

# Fit on train, transform train
X_train['Age'] = imputer.fit_transform(X_train[['Age']])

# Transform test using same statistics
X_test['Age'] = imputer.transform(X_test[['Age']])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Q: Does a preceptron require feature scaling. yes because we use gradient descent ?

### Sklearns implementation

Start at X_train_scaled, X_test_scaled, y_train, y_test and apply 

In [7]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1️⃣ Initialize the model
# You can tweak parameters like max_iter or eta0 (learning rate)
model = Perceptron(max_iter=5000, eta0=1.0, random_state=42)

# 2️⃣ Train the model
model.fit(X_train_scaled, y_train)

# 3️⃣ Make predictions
y_pred = model.predict(X_test_scaled)

# 4️⃣ Evaluate performance
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.7430

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       110
           1       0.65      0.72      0.68        69

    accuracy                           0.74       179
   macro avg       0.73      0.74      0.73       179
weighted avg       0.75      0.74      0.75       179

Confusion Matrix:
[[83 27]
 [19 50]]


In [8]:
print(X_train.columns)
print("Weights:", model.coef_)
print("Bias:", model.intercept_)


Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')
Weights: [[-3.23059232 -0.44974474 -1.42175336 -3.11571343  5.19700663  2.93418137
   2.83178782 -2.83178782 -2.03034533  0.28933346 -2.85517651]]
Bias: [-1.]


### backpropagation algorithm